## Initialise SparkSession and Load Data

In [1]:
import pyspark
conf = pyspark.SparkConf()

conf.set('spark.local.dir', 'path')
sc = pyspark.SparkContext(conf=conf)

## Connection to Google BigQuery

In [2]:
from pythonbq import pythonbq

myProject=pythonbq(
  bq_key_path='json_file',
  project_id='project_id'
)

In [3]:
SQL_CODE="""
SELECT * FROM `table.H1data`
"""
output=myProject.query(sql=SQL_CODE)

Downloading: 100%|██████████| 40060/40060 [00:10<00:00, 3799.23rows/s]


In [4]:
output

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,...,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate
0,0,4,2015,November,45,2,1,1,1,0,...,No Deposit,1,NULL,0,Transient,25.00,0,0,Check-Out,2015-11-04
1,0,5,2015,November,49,30,1,3,1,0,...,No Deposit,1,NULL,0,Transient,25.00,0,0,Check-Out,2015-12-04
2,0,11,2016,January,2,3,2,1,2,0,...,No Deposit,1,NULL,0,Transient,39.00,1,1,Check-Out,2016-01-06
3,0,258,2015,July,28,6,2,5,2,0,...,No Deposit,2,NULL,0,Contract,47.25,0,0,Check-Out,2015-07-13
4,1,292,2015,July,28,7,2,5,2,0,...,No Deposit,2,NULL,0,Contract,47.25,0,0,Canceled,2015-03-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40055,1,0,2015,July,29,17,0,1,2,0,...,No Deposit,NULL,NULL,0,Transient,8.00,0,0,Canceled,2015-07-17
40056,0,0,2015,July,28,9,0,1,1,0,...,No Deposit,NULL,NULL,0,Transient,82.00,0,0,Check-Out,2015-07-10
40057,0,0,2015,August,32,4,0,1,2,0,...,No Deposit,NULL,NULL,0,Transient,172.00,0,0,Check-Out,2015-08-05
40058,1,0,2016,February,8,15,0,0,0,0,...,No Deposit,NULL,383,0,Transient,0.00,0,0,Canceled,2016-02-15


In [5]:
# import pyspark
import pyarrow
import pandas as pd

In [6]:
type(output)

pandas.core.frame.DataFrame

In [7]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
dataset = sqlContext.createDataFrame(output)

# If reading from CSV:
# dataset= spark.read.load("H1full.csv", format="csv", header="true", inferSchema=True)

dataset
cols = dataset.columns

In [8]:
type(dataset)

pyspark.sql.dataframe.DataFrame

## Print Schema

In [9]:
dataset.printSchema()

root
 |-- IsCanceled: long (nullable = true)
 |-- LeadTime: long (nullable = true)
 |-- ArrivalDateYear: long (nullable = true)
 |-- ArrivalDateMonth: string (nullable = true)
 |-- ArrivalDateWeekNumber: long (nullable = true)
 |-- ArrivalDateDayOfMonth: long (nullable = true)
 |-- StaysInWeekendNights: long (nullable = true)
 |-- StaysInWeekNights: long (nullable = true)
 |-- Adults: long (nullable = true)
 |-- Children: long (nullable = true)
 |-- Babies: long (nullable = true)
 |-- Meal: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- MarketSegment: string (nullable = true)
 |-- DistributionChannel: string (nullable = true)
 |-- IsRepeatedGuest: long (nullable = true)
 |-- PreviousCancellations: long (nullable = true)
 |-- PreviousBookingsNotCanceled: long (nullable = true)
 |-- ReservedRoomType: string (nullable = true)
 |-- AssignedRoomType: string (nullable = true)
 |-- BookingChanges: long (nullable = true)
 |-- DepositType: string (nullable = true)
 |-- Age

## Pipeline and Data Transformation

In [10]:
# https://towardsdatascience.com/machine-learning-with-pyspark-and-mllib-solving-a-binary-classification-problem-96396065d2aa
# https://docs.databricks.com/applications/machine-learning/mllib/binary-classification-mllib-pipelines.html

from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
categoricalColumns = ["Country", "MarketSegment", "ArrivalDateMonth", "DepositType", "CustomerType"]

stages = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

## Convert label into label indices using the StringIndexer

In [11]:
label_stringIdx = StringIndexer(inputCol="IsCanceled", outputCol="label")
stages += [label_stringIdx]

## Transform all features into a vector using VectorAssembler

In [12]:
numericCols = ["LeadTime", "ArrivalDateYear", "ArrivalDateWeekNumber", "ArrivalDateDayOfMonth", "RequiredCarParkingSpaces"]
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

## GBTClassifier

In [13]:
from pyspark.ml.classification import GBTClassifier
  
partialPipeline = Pipeline().setStages(stages)
pipelineModel = partialPipeline.fit(dataset)
preppedDataDF = pipelineModel.transform(dataset)

In [14]:
gbtClassifier = GBTClassifier()
trainedModel = gbtClassifier.fit(preppedDataDF)

In [15]:
display(gbtClassifier, preppedDataDF)

GBTClassifier_99c70e6be728

DataFrame[IsCanceled: bigint, LeadTime: bigint, ArrivalDateYear: bigint, ArrivalDateMonth: string, ArrivalDateWeekNumber: bigint, ArrivalDateDayOfMonth: bigint, StaysInWeekendNights: bigint, StaysInWeekNights: bigint, Adults: bigint, Children: bigint, Babies: bigint, Meal: string, Country: string, MarketSegment: string, DistributionChannel: string, IsRepeatedGuest: bigint, PreviousCancellations: bigint, PreviousBookingsNotCanceled: bigint, ReservedRoomType: string, AssignedRoomType: string, BookingChanges: bigint, DepositType: string, Agent: string, Company: string, DaysInWaitingList: bigint, CustomerType: string, ADR: double, RequiredCarParkingSpaces: bigint, TotalOfSpecialRequests: bigint, ReservationStatus: string, ReservationStatusDate: timestamp, CountryIndex: double, CountryclassVec: vector, MarketSegmentIndex: double, MarketSegmentclassVec: vector, ArrivalDateMonthIndex: double, ArrivalDateMonthclassVec: vector, DepositTypeIndex: double, DepositTypeclassVec: vector, CustomerType

In [16]:
selectedcols = ["label", "features"] + cols
dataset = preppedDataDF.select(selectedcols)
display(dataset)

DataFrame[label: double, features: vector, IsCanceled: bigint, LeadTime: bigint, ArrivalDateYear: bigint, ArrivalDateMonth: string, ArrivalDateWeekNumber: bigint, ArrivalDateDayOfMonth: bigint, StaysInWeekendNights: bigint, StaysInWeekNights: bigint, Adults: bigint, Children: bigint, Babies: bigint, Meal: string, Country: string, MarketSegment: string, DistributionChannel: string, IsRepeatedGuest: bigint, PreviousCancellations: bigint, PreviousBookingsNotCanceled: bigint, ReservedRoomType: string, AssignedRoomType: string, BookingChanges: bigint, DepositType: string, Agent: string, Company: string, DaysInWaitingList: bigint, CustomerType: string, ADR: double, RequiredCarParkingSpaces: bigint, TotalOfSpecialRequests: bigint, ReservationStatus: string, ReservationStatusDate: timestamp]

## Train-Test Split

In [17]:
(trainingData, testData) = dataset.randomSplit([0.8, 0.2], seed=100)
print(trainingData.count())
print(testData.count())

32101
7959


In [18]:
gbtModel = gbtClassifier.fit(trainingData)

In [19]:
predictions = gbtModel.transform(testData)
predictions

DataFrame[label: double, features: vector, IsCanceled: bigint, LeadTime: bigint, ArrivalDateYear: bigint, ArrivalDateMonth: string, ArrivalDateWeekNumber: bigint, ArrivalDateDayOfMonth: bigint, StaysInWeekendNights: bigint, StaysInWeekNights: bigint, Adults: bigint, Children: bigint, Babies: bigint, Meal: string, Country: string, MarketSegment: string, DistributionChannel: string, IsRepeatedGuest: bigint, PreviousCancellations: bigint, PreviousBookingsNotCanceled: bigint, ReservedRoomType: string, AssignedRoomType: string, BookingChanges: bigint, DepositType: string, Agent: string, Company: string, DaysInWaitingList: bigint, CustomerType: string, ADR: double, RequiredCarParkingSpaces: bigint, TotalOfSpecialRequests: bigint, ReservationStatus: string, ReservationStatusDate: timestamp, rawPrediction: vector, probability: vector, prediction: double]

In [20]:
selected = predictions.select("label", "prediction", "probability")
display(selected)

DataFrame[label: double, prediction: double, probability: vector]

## Model Evaluation

In [21]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

0.9123967272115422

In [22]:
evaluator.getMetricName()

'areaUnderROC'

In [23]:
print(gbtModel.explainParams())

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n]. (default: all)
featuresCol: features column name (default: features)
impurity: Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini (undefined)
labelCol: label column name (default: label)
lossType: Loss function which GBT tries to minimize (case-insensitive). Supported options: logistic (default: logistic)
ma

In [24]:
predictions = gbtModel.transform(testData)

In [25]:
evaluator.evaluate(predictions)

0.9123967272115416

In [26]:
selected = predictions.select("label", "prediction", "probability")
display(selected)

DataFrame[label: double, prediction: double, probability: vector]

In [27]:
type(selected)

pyspark.sql.dataframe.DataFrame

In [28]:
selected.toPandas().to_csv('h1predictions.csv')

## Predictions on Test Data

In [29]:
mydata=pd.read_csv("h1predictions.csv")
mydata

,Unnamed: 0,label,prediction,probability
0,0,0.0,1.0,"[0.3328517247925773,0.6671482752074227]"
1,1,0.0,1.0,"[0.33385578033961344,0.6661442196603866]"
2,2,0.0,1.0,"[0.33385578033961344,0.6661442196603866]"
3,3,0.0,1.0,"[0.3295129995457048,0.6704870004542952]"
4,4,0.0,1.0,"[0.22810083419630225,0.7718991658036978]"
...,...,...,...,...
7954,7954,1.0,0.0,"[0.593856422149016,0.406143577850984]"
7955,7955,1.0,0.0,"[0.870767922996301,0.129232077003699]"
7956,7956,1.0,0.0,"[0.7108949915741888,0.2891050084258112]"
7957,7957,1.0,0.0,"[0.6716583759429362,0.3283416240570638]"


In [30]:
import numpy as np
from numpy.random import seed
seed(1)
from sklearn.metrics import classification_report,confusion_matrix

In [31]:
label=mydata['label']
prediction=mydata['prediction']

In [32]:
print(confusion_matrix(label,prediction))
print(classification_report(label,prediction))

[[5242  530]
 [ 752 1435]]
              precision    recall  f1-score   support

         0.0       0.87      0.91      0.89      5772
         1.0       0.73      0.66      0.69      2187

    accuracy                           0.84      7959
   macro avg       0.80      0.78      0.79      7959
weighted avg       0.83      0.84      0.84      7959

